&emsp;
## &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&emsp;&emsp;&emsp;&emsp;**Лабораторная работа №5**

###  Постановка задачи

* Реализовать 5 процессов, в каждом по три нити с фильтром из 4 лабораторной работы и общим процесс-безопасным журналом, l log уровнями, с info, Debug, прерываниями. Работа с журналом осуществляется в фоновом режиме.
* Сообщения в журнале в формате: <начало\окончание>: <номер нити\процесса> <главный процесс - окончание>

### Филтер
Фильтр, выполняющий инверсию каждого  слова строки.


In [3]:
import random
from time import sleep
from threading import current_thread
from multiprocessing import current_process
from threading import Thread
from datetime import datetime
from multiprocessing import Process
from multiprocessing import Queue
from logging.handlers import QueueHandler
import logging


def logger_process(queue):
    logger = logging.getLogger('app')
    if logger.hasHandlers():
        logger.handlers.clear()
    logger.addHandler(logging.StreamHandler())
    logger.setLevel(logging.DEBUG)
    while True:
        message = queue.get()
        if message is None:
            break
        logger.handle(message)


def thread_task(str):
    logger = logging.getLogger('app')
    letters = str.split(" ")
    for i in range(len(letters)):
            print(letters[i][len(letters[i])::-1])
        
    process_name = current_process().name
    thread_name = current_thread().name
    logger.info(
        f'Thread {thread_name} in process {process_name} start at {datetime.now()}')
    # МОЯ ФУНКЦИЯ
    logger.info('--------')
    for letter in letters:
        logger.info(letter)
    logger.info('--------')
    #
    logger.info(
        f'Thread {thread_name} in process {process_name} done at {datetime.now()}')


def process_task(queue):
    process_name = current_process().name
    logger = logging.getLogger('app')
    if logger.hasHandlers():
        logger.handlers.clear()
    logger.addHandler(QueueHandler(queue))
    logger.setLevel(logging.DEBUG)
    logger.info(f'Process {process_name} start at time {datetime.now()}')
    threads = [Thread(target=thread_task, args=("Hello world Abob lad ABC",))
               for _ in range(3)]
    for thread in threads:
        thread.start()
        sleep(0.01)
    for thread in threads:
        thread.join()
    logger.info(f'Process {process_name} done at time {datetime.now()}')


if __name__ == '__main__':
    queue = Queue()
    logger = logging.getLogger('app')
    logger.addHandler(logging.StreamHandler())
    logger.setLevel(logging.DEBUG)
    logger_p = Process(target=logger_process, args=(queue,), daemon=True)
    logger_p.start()
    logger.info('Main process started.')
    processes = [Process(target=process_task, args=(queue,)) for _ in range(5)]
    for process in processes:
        process.start()
        sleep(0.1)
    for process in processes:
        process.join()
    logger.info('Main process done.')
    queue.put(None)


Main process started.
Main process started.
Main process started.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "<string>", line 1, in <module>
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
At